In [4]:
from src.data.imp_datamodule import IMP_train
from src.utils.clustering_utils import *
from src.models.components.clustering import get_umap, get_kmeans
import json
import os
import warnings
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageFile

import hydra
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
from omegaconf import DictConfig, OmegaConf
from torch.optim.lr_scheduler import (
    CosineAnnealingLR,
)
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from transformers import AutoTokenizer, AutoImageProcessor

In [5]:
class IMP_train(Dataset):
    def __init__(self, annotation_path, image_path, preprocess, ratio=0.1):
        self.annotations = json.load(open(annotation_path))
        self.annotations = self.annotations[: int(len(self.annotations) * ratio)]
        self.image_path = image_path
        self.vis_processors = preprocess

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        raw_image = Image.open(
            os.path.join(self.image_path, self.annotations[idx]["image"])
        ).convert("RGB")
        image_input = self.vis_processors(raw_image, return_tensors="pt")
        if "pixel_values" in image_input:
            image_input["pixel_values"] = image_input["pixel_values"].squeeze()

        raw_text = self.annotations[idx]["caption"]

        return image_input, raw_text

In [3]:
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)

preprocess = AutoImageProcessor.from_pretrained("openai/clip-vit-base-patch32")
tokenizer = AutoTokenizer.from_pretrained("openai/clip-vit-base-patch32")

train_dataset = IMP_train(
        annotation_path="/data/SSD/coco/annotations/coco_karpathy_train.json",
        image_path="/data/SSD/coco/images",
        preprocess=preprocess,
        ratio=1,
    )
train_dataloader = DataLoader(
    train_dataset,
    batch_size=1024,
    shuffle=True,
    num_workers=8,
)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [ ]:
for batch_id, batch in enumerate(train_dataloader):
    